<a href="https://colab.research.google.com/github/Yashchauhan23/Deep_Learning/blob/main/EncoderAndDecoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000
data_path = '/content/fra.txt'

**Vectorize the data**

In [42]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
     lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) -1)]:
    input_text, target_text, _ = line.split('\t')
    target_text = '\t' +target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
             input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
             target_characters.add(char)
    

In [43]:
input_characters

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é'}

In [44]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [45]:
print('Number of samples: ',len(input_texts))
print('Number of unique input tokens: ',num_encoder_tokens)
print('Number of unique output tokens: ',num_decoder_tokens)
print('Max Sequence length for inputs: ',max_encoder_seq_length)
print('Max Sequence length for outputs: ',max_decoder_seq_length)

Number of samples:  10000
Number of unique input tokens:  71
Number of unique output tokens:  93
Max Sequence length for inputs:  15
Max Sequence length for outputs:  59


In [46]:
input_token_index = dict(
    [(char,i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char,i) for i, char in enumerate(target_characters)])

In [47]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 'é': 70}

In [48]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [49]:

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
            for t, char in enumerate(input_text):
                encoder_input_data[i, t, input_token_index[char]] = 1.
            encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
            for t, char in enumerate(target_text):
                # decoder_target_data is ahead of decoder_input_data by one timestep
                decoder_input_data[i, t, target_token_index[char]] = 1.
                if t > 0:
                    # decoder_target_data will be ahead by one timestep
                    # and will not include the start character.
                    decoder_target_data[i, t - 1, target_token_index[char]] = 1.
            decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
            decoder_target_data[i, t:, target_token_index[' ']] = 1.
    

In [50]:
encoder_input_data[0].shape

(15, 71)

In [51]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder =  LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [52]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                               initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [53]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
            
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
                                              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size = batch_size,
          epochs = epochs,
          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 5s 19ms/step - loss: 1.1549 - accuracy: 0.7357 - val_loss: 1.0364 - val_accuracy: 0.7129
Epoch 2/100
125/125 [==============================] - 2s 12ms/step - loss: 0.8292 - accuracy: 0.7770 - val_loss: 0.8373 - val_accuracy: 0.7665
Epoch 3/100
125/125 [==============================] - 1s 12ms/step - loss: 0.6648 - accuracy: 0.8154 - val_loss: 0.7097 - val_accuracy: 0.7925
Epoch 4/100
125/125 [==============================] - 2s 12ms/step - loss: 0.5743 - accuracy: 0.8333 - val_loss: 0.6408 - val_accuracy: 0.8122
Epoch 5/100
125/125 [==============================] - 2s 12ms/step - loss: 0.5257 - accuracy: 0.8464 - val_loss: 0.5863 - val_accuracy: 0.8303
Epoch 6/100
125/125 [==============================] - 2s 12ms/step - loss: 0.4898 - accuracy: 0.8563 - val_loss: 0.5636 - val_accuracy: 0.8361
Epoch 7/100
125/125 [==============================] - 2s 13ms/step - loss: 0.4616 - accuracy: 0.8632 - val_loss: 0.5340 - val_accuracy:

In [56]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

reverse_input_char_index = dict(
    (i,char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
     (i,char) for char, i in input_token_index.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

for seq_index in range(100):
  input_seq = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence=decode_sequence(input_seq)
  print('-')
  print('Input Sentence: ',input_texts[seq_index])
  print('Decoded Sentence: ',decoded_sentence)
        

-
Input Sentence:  Go.
Decoded Sentence:  Wb"$!"""""""""""""""""""""""""""""""""""""""""""""""""""""""
-
Input Sentence:  Go.
Decoded Sentence:  Wb"$!"""""""""""""""""""""""""""""""""""""""""""""""""""""""
-
Input Sentence:  Go.
Decoded Sentence:  Wb"$!"""""""""""""""""""""""""""""""""""""""""""""""""""""""
-
Input Sentence:  Hi.
Decoded Sentence:  Tbmvu1!"""""""""""""""""""""""""""""""""""""""""""""""""""""
-
Input Sentence:  Hi.
Decoded Sentence:  Tbmvu1!"""""""""""""""""""""""""""""""""""""""""""""""""""""
-
Input Sentence:  Run!
Decoded Sentence:  Gvzfé"$!""""""""""""""""""""""""""""""""""""""""""""""""""""
-
Input Sentence:  Run!
Decoded Sentence:  Gvzfé"$!""""""""""""""""""""""""""""""""""""""""""""""""""""
-
Input Sentence:  Run!
Decoded Sentence:  Gvzfé"$!""""""""""""""""""""""""""""""""""""""""""""""""""""
-
Input Sentence:  Run!
Decoded Sentence:  Gvzfé"$!""""""""""""""""""""""""""""""""""""""""""""""""""""
-
Input Sentence:  Run!
Decoded Sentence:  Gvzfé"$!""""""""""""""""""

KeyError: ignored